# Introduction

In this demo, we will demostrate how to fine-tune a pre-trained model to do text classification.

# Load Sample Data

In [ ]:
import pandas as pd
from datasets import Dataset

sample_data = pd.read_csv('data/train.csv')[['full_text', 'cohesion']]
sample_data.columns = ['text', 'label']
sample_data

In [ ]:
dataset = Dataset.from_pandas(sample_data, preserve_index=False) 
dataset = dataset.train_test_split(test_size=0.3) 
dataset

# Fine-Tune the Model

In [ ]:
from transformers import  AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')

def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)  


In [ ]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

In [ ]:
# Metrics Function

from sklearn.metrics import mean_squared_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

In [ ]:
# Train the model

from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)
trainer.train()

# Save and Load the Model and Tokenizer

In [ ]:
# save the model/tokenizer
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")

# load the model/tokenizer
from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

# Use the Model

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model)


def pipeline_prediction(text):
    df=pd.DataFrame({'text':[text]})
    dataset = Dataset.from_pandas(df,preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred[0][0])

In [ ]:
raw_pred, labels, metrics = trainer.predict(tokenized_datasets['test'])

In [ ]:
raw_pred

In [ ]:
labels

In [ ]:
metrics